Alhassan, Abdulaziz and Sevtsuk, Andres, Madina Python Package: Scalable Urban Network Analysis for Modeling Pedestrian and Bicycle Trips in Cities. Available at SSRN: https://ssrn.com/abstract=4748255 or http://dx.doi.org/10.2139/ssrn.4748255

In [1]:
#import madina as md
import geopandas as gpd
import matplotlib.pyplot as plt

#nyc = md.Zonal()

In [2]:
nycmap = gpd.read_file('cut.shp')

In [9]:
nycmap

FID_dcp_dc  cartodb_id        borough   feat_type     feat_statu  \
0            8578        7836  Staten Island   Mapped_St       Paper_St   
1           22820       22677  Staten Island   Mapped_St       Paper_St   
2            8578        7836  Staten Island   Mapped_St       Paper_St   
3           38150       38073  Staten Island   Mapped_St        City_St   
4           38149       38072  Staten Island   Mapped_St        City_St   
...           ...         ...            ...         ...            ...   
88744       18485       18282          Bronx   Mapped_St        City_St   
88745       21374       21205          Bronx   Mapped_St        City_St   
88746       30517       30392          Bronx   Mapped_St        City_St   
88747       18485       18282          Bronx   Mapped_St        City_St   
88748       17597       17372          Bronx  Not_mapped  Way_on_record   

              street_nm honorarynm old_st_nm streetwidt route_type  \
0           Surf Avenue       None       NaN         60     Mjr_st   
1       Brighton Street       None       NaN         70    Gen_use   
2           Surf Avenue       None       NaN         60     Mjr_st   
3         Billop Avenue       None       NaN         70    Gen_use   
4         Billop Avenue       None       NaN         70    Gen_use   
...                 ...        ...       ...        ...        ...   
88744   West 261 Street       None       NaN         60    Gen_use   
88745   West 261 Street       None       NaN         60    Gen_use   
88746  Riverdale Avenue       None       NaN        100     Mjr_st   
88747   West 261 Street       None       NaN         60    Gen_use   
88748        River Road       None       NaN        n/a      Other   

       roadwaytyp     build_stat record_st paper_st stair_st cco_st  \
0      Surface_ST     Unimproved       NaN        Y      NaN    NaN   
1      Surface_ST     Unimproved       NaN        Y      NaN    NaN   
2      Surface_ST     Unimproved       NaN        Y      NaN    NaN   
3      Surface_ST       Improved       NaN      NaN      NaN    NaN   
4      Surface_ST       Improved       NaN      NaN      NaN    NaN   
...           ...            ...       ...      ...      ...    ...   
88744  Surface_ST  Part_improved       NaN      NaN      NaN    NaN   
88745  Surface_ST  Part_improved       NaN      NaN      NaN    NaN   
88746  Surface_ST       Improved       NaN      NaN      NaN    NaN   
88747  Surface_ST  Part_improved       NaN      NaN      NaN    NaN   
88748  Surface_ST       Improved       NaN      NaN      NaN    NaN   

      marg_wharf edit_date  Shape_Leng  \
0            NaN       NaN    0.000921   
1            NaN       NaN    0.000484   
2            NaN       NaN    0.000913   
3            NaN       NaN    0.000114   
4            NaN       NaN    0.000900   
...          ...       ...         ...   
88744        NaN       NaN    0.001292   
88745        NaN       NaN    0.000116   
88746        NaN       NaN    0.000649   
88747        NaN       NaN    0.000724   
88748        NaN       NaN    0.000198   

                                                geometry  
0      LINESTRING (-74.24120 40.49795, -74.24106 40.4...  
1      LINESTRING (-74.24135 40.49841, -74.24120 40.4...  
2      LINESTRING (-74.24038 40.49837, -74.23956 40.4...  
3      LINESTRING (-74.24448 40.49893, -74.24437 40.4...  
4      LINESTRING (-74.24437 40.49895, -74.24349 40.4...  
...                                                  ...  
88744  LINESTRING (-73.91061 40.91285, -73.91057 40.9...  
88745  LINESTRING (-73.91069 40.91293, -73.91066 40.9...  
88746  LINESTRING (-73.90258 40.91250, -73.90246 40.9...  
88747  LINESTRING (-73.91057 40.91363, -73.91071 40.9...  
88748  LINESTRING (-73.91046 40.91360, -73.91046 40.9...  

[88749 rows x 20 columns]

In [24]:
fig, ax = plt.subplots(figsize=(20, 20))  # Adjust the size as needed
nycmap.plot(ax=ax)

# Optional: Customize the plot
ax.set_title('Map of Streets')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Show the plot
plt.show()

In [5]:
import pandas as pd

import itertools
from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString
from operator import itemgetter

In [6]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt



In [7]:
ox.config(use_cache=True, log_console=True)

# Fetch the street network for New York City
G = ox.graph_from_place('New York City, New York, USA', network_type='drive')


<ipython-input-7-8c838bf39454>:1: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(use_cache=True, log_console=True)


In [8]:
crash_data = pd.read_csv('Motor_Crashes_20240427.csv')
crash_data.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
gdf_crashes = gpd.GeoDataFrame(crash_data, geometry=gpd.points_from_xy(crash_data['LONGITUDE'], crash_data['LATITUDE']), crs='EPSG:4326')
gdf_crashes = gdf_crashes.to_crs(ox.settings.default_crs) 

edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
edges_sindex = edges.sindex


<ipython-input-8-9cfbb66095fe>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  crash_data = pd.read_csv('Motor_Crashes_20240427.csv')


In [10]:
gdf_crashes

CRASH DATE CRASH TIME        BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
3        09/11/2021       9:35       BROOKLYN  11208.0  40.667202 -73.866500   
4        12/14/2021       8:13       BROOKLYN  11233.0  40.683304 -73.917274   
6        12/14/2021      17:05            NaN      NaN  40.709183 -73.956825   
7        12/14/2021       8:17          BRONX  10475.0  40.868160 -73.831480   
8        12/14/2021      21:10       BROOKLYN  11207.0  40.671720 -73.897100   
...             ...        ...            ...      ...        ...        ...   
2083222  04/23/2024      19:32         QUEENS  11101.0  40.744560 -73.930760   
2083223  04/20/2024       3:48  STATEN ISLAND  10312.0  40.546530 -74.180435   
2083224  04/20/2024      22:54      MANHATTAN  10031.0  40.820095 -73.955086   
2083225  04/23/2024       7:33         QUEENS  11693.0  40.587740 -73.809875   
2083226  04/22/2024      12:40       BROOKLYN  11207.0  40.671980 -73.886410   

                        LOCATION              ON STREET NAME  \
3          (40.667202, -73.8665)                         NaN   
4        (40.683304, -73.917274)             SARATOGA AVENUE   
6        (40.709183, -73.956825)  BROOKLYN QUEENS EXPRESSWAY   
7          (40.86816, -73.83148)                         NaN   
8           (40.67172, -73.8971)                         NaN   
...                          ...                         ...   
2083222    (40.74456, -73.93076)                   34 STREET   
2083223   (40.54653, -74.180435)                ARDEN AVENUE   
2083224  (40.820095, -73.955086)                    BROADWAY   
2083225   (40.58774, -73.809875)    ROCKAWAY BEACH BOULEVARD   
2083226    (40.67198, -73.88641)              BELMONT AVENUE   

           CROSS STREET NAME              OFF STREET NAME  ...  \
3                        NaN      1211      LORING AVENUE  ...   
4             DECATUR STREET                          NaN  ...   
6                        NaN                          NaN  ...   
7                        NaN  344       BAYCHESTER AVENUE  ...   
8                        NaN      2047      PITKIN AVENUE  ...   
...                      ...                          ...  ...   
2083222     QUEENS BOULEVARD                          NaN  ...   
2083223  DRUMGOOLE ROAD EAST                          NaN  ...   
2083224      WEST 135 STREET                          NaN  ...   
2083225      BEACH 84 STREET                          NaN  ...   
2083226        JEROME STREET                          NaN  ...   

         CONTRIBUTING FACTOR VEHICLE 3  CONTRIBUTING FACTOR VEHICLE 4  \
3                                  NaN                            NaN   
4                                  NaN                            NaN   
6                                  NaN                            NaN   
7                                  NaN                            NaN   
8                                  NaN                            NaN   
...                                ...                            ...   
2083222                            NaN                            NaN   
2083223                            NaN                            NaN   
2083224                            NaN                            NaN   
2083225                            NaN                            NaN   
2083226                    Unspecified                            NaN   

         CONTRIBUTING FACTOR VEHICLE 5  COLLISION_ID  \
3                                  NaN       4456314   
4                                  NaN       4486609   
6                                  NaN       4486555   
7                                  NaN       4486660   
8                                  NaN       4487074   
...                                ...           ...   
2083222                            NaN       4719738   
2083223                            NaN       4719810   
2083224                            NaN       4719839   
2083225                            NaN       4719

In [12]:
import numpy as np



In [14]:
def ckdnearest(gpd1, gpd2, gpd2_cols=['street_nm']):
    # Extract coordinates from LineStrings in gpd2 and create a flat list of coordinates and their original index
    B = [np.array(geom.coords) for geom in gpd2.geometry.to_list()]
    B_ix = tuple(itertools.chain.from_iterable(
        [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))
    B = np.concatenate(B)

    ckd_tree = cKDTree(B)

    # query
    A = np.vstack(gpd1.geometry.apply(lambda geom: np.array(geom.coords[0])))
    dist, idx = ckd_tree.query(A, k=1)
    idx = itemgetter(*idx)(B_ix) if len(gpd1) > 1 else itemgetter(idx)(B_ix)  # handle single point case

    gpd_result = pd.concat(
        [gpd1.reset_index(drop=True), 
         gpd2.loc[idx, gpd2_cols].reset_index(drop=True), 
         pd.Series(dist, name='dist')], axis=1)
    return gpd_result


result = ckdnearest(gdf_crashes, nycmap)
print(result)

         CRASH DATE CRASH TIME        BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
0        09/11/2021       9:35       BROOKLYN  11208.0  40.667202 -73.866500   
1        12/14/2021       8:13       BROOKLYN  11233.0  40.683304 -73.917274   
2        12/14/2021      17:05            NaN      NaN  40.709183 -73.956825   
3        12/14/2021       8:17          BRONX  10475.0  40.868160 -73.831480   
4        12/14/2021      21:10       BROOKLYN  11207.0  40.671720 -73.897100   
...             ...        ...            ...      ...        ...        ...   
1849068  04/23/2024      19:32         QUEENS  11101.0  40.744560 -73.930760   
1849069  04/20/2024       3:48  STATEN ISLAND  10312.0  40.546530 -74.180435   
1849070  04/20/2024      22:54      MANHATTAN  10031.0  40.820095 -73.955086   
1849071  04/23/2024       7:33         QUEENS  11693.0  40.587740 -73.809875   
1849072  04/22/2024      12:40       BROOKLYN  11207.0  40.671980 -73.886410   

                        LOCATION       

In [17]:
result

CRASH DATE CRASH TIME        BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
0        09/11/2021       9:35       BROOKLYN  11208.0  40.667202 -73.866500   
1        12/14/2021       8:13       BROOKLYN  11233.0  40.683304 -73.917274   
2        12/14/2021      17:05            NaN      NaN  40.709183 -73.956825   
3        12/14/2021       8:17          BRONX  10475.0  40.868160 -73.831480   
4        12/14/2021      21:10       BROOKLYN  11207.0  40.671720 -73.897100   
...             ...        ...            ...      ...        ...        ...   
1849068  04/23/2024      19:32         QUEENS  11101.0  40.744560 -73.930760   
1849069  04/20/2024       3:48  STATEN ISLAND  10312.0  40.546530 -74.180435   
1849070  04/20/2024      22:54      MANHATTAN  10031.0  40.820095 -73.955086   
1849071  04/23/2024       7:33         QUEENS  11693.0  40.587740 -73.809875   
1849072  04/22/2024      12:40       BROOKLYN  11207.0  40.671980 -73.886410   

                        LOCATION              ON STREET NAME  \
0          (40.667202, -73.8665)                         NaN   
1        (40.683304, -73.917274)             SARATOGA AVENUE   
2        (40.709183, -73.956825)  BROOKLYN QUEENS EXPRESSWAY   
3          (40.86816, -73.83148)                         NaN   
4           (40.67172, -73.8971)                         NaN   
...                          ...                         ...   
1849068    (40.74456, -73.93076)                   34 STREET   
1849069   (40.54653, -74.180435)                ARDEN AVENUE   
1849070  (40.820095, -73.955086)                    BROADWAY   
1849071   (40.58774, -73.809875)    ROCKAWAY BEACH BOULEVARD   
1849072    (40.67198, -73.88641)              BELMONT AVENUE   

           CROSS STREET NAME              OFF STREET NAME  ...  \
0                        NaN      1211      LORING AVENUE  ...   
1             DECATUR STREET                          NaN  ...   
2                        NaN                          NaN  ...   
3                        NaN  344       BAYCHESTER AVENUE  ...   
4                        NaN      2047      PITKIN AVENUE  ...   
...                      ...                          ...  ...   
1849068     QUEENS BOULEVARD                          NaN  ...   
1849069  DRUMGOOLE ROAD EAST                          NaN  ...   
1849070      WEST 135 STREET                          NaN  ...   
1849071      BEACH 84 STREET                          NaN  ...   
1849072        JEROME STREET                          NaN  ...   

         CONTRIBUTING FACTOR VEHICLE 5  COLLISION_ID  \
0                                  NaN       4456314   
1                                  NaN       4486609   
2                                  NaN       4486555   
3                                  NaN       4486660   
4                                  NaN       4487074   
...                                ...           ...   
1849068                            NaN       4719738   
1849069                            NaN       4719810   
1849070                            NaN       4719839   
1849071                            NaN       4719674   
1849072                            NaN       4719792   

                         VEHICLE TYPE CODE 1  \
0                                      Sedan   
1                                        NaN   
2                                      Sedan   
3                                      Sedan   
4                                      Sedan   
...                                      ...   
1849068  Station Wagon/Sport Utility Vehicle   
1849069  Station Wagon/Sport Utility Vehicle   
1849070                                Moped   
1849071                                Sedan   
1849072                                Sedan   

                         VEHICLE TYPE CODE 2  VEHICLE TYPE CODE 3  \
0                                        NaN                  NaN   
1                                        NaN                  NaN   
2                       Tract

In [14]:

#nyc.load_layer(
#    name='nycstreet',
#    source = nycmap
#)

In [15]:
#nyc['nycstreet'].gdf

In [16]:
#nyc.create_street_network(source_layer="nycstreet")

In [17]:
#nyc.network.nodes.head(5)